In [ ]:
### per CVAT Bilder taggen, export dataset nicht als yolo sondern "CVAT for images 1.1" ###

### Das Dataset entpacken, und splitten ###
### Ist kein Label vorhanden, nutze 'under_water' ###

def extract_and_split(zip_file, output_folder_var):
    try:
        with zipfile.ZipFile(zip_file) as z:
            z.extractall(output_folder_var)
            print("Extracted all files")
    except:
        print("Invalid file")
        
    if not os.path.exists(output_folder_var):
        print("Output folder does not exist")
        return
        
    # Pfad zur XML-Datei
    xml_file = os.path.join(output_folder_var, 'annotations.xml')

    # Pfad zum Ordner für die Bilder
    img_dir = os.path.join(output_folder_var, 'images/default')

    # Liste der Klassen
    classes = set()

    # Pfad zum Unterordner für die Textdateien
    txt_dir = os.path.join(output_folder_var, 'images/labels')

    # Schleife durch die XML-Datei, um alle Klassen zu sammeln
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for image in root.findall('image'):
        for tag in image.findall('tag'):
            label = tag.get('label')
            classes.add(label)

    # Schleife durch alle Bilder in der XML-Datei
    for image in root.findall('image'):
        # Extrahieren des Dateinamens
        file_name = image.get('name')
        # Überprüfen, ob das Bild vorhanden ist
        img_file_path = os.path.join(img_dir, file_name)
        if not os.path.exists(img_file_path):
            print(f"{img_file_path} not found, skipping")
            continue
        # Extrahieren des Bildbreite und -höhe
        width = int(image.get('width'))
        height = int(image.get('height'))
        # Schleife durch alle Tags des Bildes, um die Klasse zu bestimmen
        class_label = 'under_water' # Wenn kein Label gefunden wird, wird "under_water" verwendet
        for tag in image.findall('tag'):
            label = tag.get('label')
            if label in classes:
                class_label = label
                break
        # Erstellen des Zielordners, falls er nicht vorhanden ist
        class_dir = os.path.join(txt_dir, class_label)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        # Kopieren der Bild- und Textdateien in den Zielordner
        txt_file_path = os.path.join(class_dir, os.path.splitext(file_name)[0] + '.txt')
        if class_label != 'under_water':
            shutil.copy(img_file_path, class_dir)
            with open(txt_file_path, 'w') as f:
                f.write(f"{class_label}\n")
                print(f"Wrote label {class_label} for {file_name}")
        else:
            print(f"No label found for {file_name}, copying to {class_dir}")
            # Kopieren der Bild- und Textdateien in den Zielordner für "under_water"
            shutil.copy(img_file_path, class_dir)
            shutil.copy(img_file_path, os.path.join(txt_dir, class_label))
            with open(txt_file_path, 'w') as f:
                f.write(f"{class_label}\n")
                print(f"Wrote label {class_label} for {file_name}")
                
    # Splitten der Daten in Train-, Validierungs- und Testdaten
    input_folder = txt_dir
    output_folder = os.path.join(output_folder_var, 'images/split')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.5, 0.3, 0.2))
    print("Data split complete")
    
extract_and_split('Pfad/zu/zip', 'Pfad/zu/output')

In [ ]:
### trainiere ein Model auf das Datenset ### 

from ultralytics import YOLO
import cv2

# Laden des YOLOv8-Modells
model = YOLO("customize/model/yolov8x-cls.yaml") #lade Model

# Trainieren des Modells
model.train(data="customize/data/tag_cvat.yaml", epochs=20, batch=4, device=0, seed=42, project='Zielpfad', name='name', pretrained=True, save=True, workers=2, imgsz=640, task='classify')

In [ ]:
### Vorhersage auf unbekannte Ausgeschnittene Delfin Bilder ###
import cv2
from ultralytics import YOLO

model = YOLO("Pfad/zu/trainierten/Model/best.pt")  # load a custom model
model.predict("Pfad/zu/Bilderordner", save=False, conf=0.5)